In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=fa3d3d4dd51850bcc2e77cd25e2f7a228b6916d7eb1af3e40580be72b24ad196
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [128]:
from pyspark.sql.functions import array_contains, col, udf, split, trim, explode
from pyspark.sql.types import StringType, StructType, StructField

In [105]:

df_data = spark.read.json('/content/1.json')

In [91]:
print(f"Total de filas: {df_data.count()}")

Total de filas: 275001


In [34]:
df_estados = spark.read.option("delimiter", ";").option("header", "true").csv('/content/estados_usa.csv')

In [ ]:
df_estados.show(50)

In [106]:
# Convertir el DataFrame a Pandas y luego a un diccionario
estados_dict = df_estados.select("nombre_corto", "nombre_largo").toPandas().set_index("nombre_corto").to_dict()["nombre_largo"]

In [107]:
df_data = df_data.drop('relative_results', 'num_of_reviews', 'description', 'url', 'MISC', 'hours')

In [108]:
df_data = df_data.dropDuplicates(['gmap_id'])
df_data = df_data.dropna(subset=['address'])
df_data = df_data.dropna(subset=['category'])

In [109]:
# Crear una nueva columna que indique si la categoría contiene 'Restaurant'
df_data = df_data.withColumn('Es_Restaurant', array_contains(col('category'), 'Restaurant'))

In [110]:
df_data = df_data.filter(col('Es_Restaurant') == True)

In [111]:
df_data.show(3)

+--------------------+----------+--------------------+--------------------+----------+------------+--------------------+-----+--------------------+-------------+
|             address|avg_rating|            category|             gmap_id|  latitude|   longitude|                name|price|               state|Es_Restaurant|
+--------------------+----------+--------------------+--------------------+----------+------------+--------------------+-----+--------------------+-------------+
|Rob's Lunch Box D...|       4.8|[Restaurant, Rest...|0x407636ffaaab494...|38.3551241|-122.7086001|Rob's Lunch Box D...| NULL|  Permanently closed|         true|
|Shell, 529 US-1, ...|       4.5|[Gas station, ATM...|0x4ca8ad3fffffffa...|44.9523393| -67.1744369|               Shell|   $$|            Open now|         true|
|New Meadows Seafo...|       4.8|[Seafood market, ...|0x4cad85e6d5f862d...|43.9280587| -69.8585195|New Meadows Seafo...| NULL|Closed ⋅ Opens 10...|         true|
+--------------------+------

In [112]:
print(f"Total de filas: {df_data.count()}")

Total de filas: 4867


In [113]:
df_data= df_data.fillna({'price':'SIN DATO', 'state':'SIN DATO'})     # Se imputan los valores nulos a 'SIN DATO'

In [114]:
# Definir la función UDF
def ext_ciudad_estado(dir, estados_dict):
    ciudad = "SIN DATO"
    estado = "SIN DATO"
    if len(str(dir)) > 10:
        lista = str(dir).split(',')
        if len(lista) > 2:
            codigo = lista[-1][1:3]
            estado = estados_dict.get(codigo, "SIN DATO")
            ciudad = lista[-2].strip() if estado != "SIN DATO" else "SIN DATO"
    return ciudad, estado

In [115]:
# Registrar la UDF usando el diccionario
@udf(StructType([StructField("ciudad", StringType(), True), StructField("estado", StringType(), True)]))
def ext_ciudad_estado_udf(dir):
    return ext_ciudad_estado(dir, estados_dict)

In [116]:
# Aplicar la UDF al DataFrame
df_data = df_data.withColumn("ciudad_estado", ext_ciudad_estado_udf(df_data["address"]))
df_data = df_data.withColumn("ciudad", col("ciudad_estado").getItem("ciudad")) \
                 .withColumn("estado", col("ciudad_estado").getItem("estado")) \
                 .drop("ciudad_estado")

In [122]:
df_data = df_data.withColumn('estado', trim(col('estado')))

In [126]:
lista_estados= ['Florida', 'Pennsylvania', 'Tennessee', 'California', 'Texas', 'New York']
# Filtrar el DataFrame
df_data = df_data.filter(col('estado').isin(lista_estados))

In [127]:
df_data.show(3)

+--------------------+----------+--------------------+--------------------+----------+-------------------+--------------------+--------+------------------+-------------+------------+----------+
|             address|avg_rating|            category|             gmap_id|  latitude|          longitude|                name|   price|             state|Es_Restaurant|      ciudad|    estado|
+--------------------+----------+--------------------+--------------------+----------+-------------------+--------------------+--------+------------------+-------------+------------+----------+
|Rob's Lunch Box D...|       4.8|[Restaurant, Rest...|0x407636ffaaab494...|38.3551241|       -122.7086001|Rob's Lunch Box D...|SIN DATO|Permanently closed|         true|Rohnert Park|California|
|The Hungry Hiker,...|       4.0|        [Restaurant]|0x4ccae17c7ed5a88...|44.1827946|        -73.9638427|    The Hungry Hiker|SIN DATO|          SIN DATO|         true| Lake Placid|  New York|
|Yaadgar Restauran...|       4

In [132]:
# Seleccionar las columnas necesarias
df_category = df_data.select('gmap_id', 'category')

# Explotar la columna 'category'
df_category = df_category.withColumn('category', explode(col('category')))

In [139]:
df_category = df_category.withColumnRenamed('gmap_id', 'business_id') \
                          .withColumnRenamed('category', 'category_name')


In [141]:
df_category.show(3)

+--------------------+------------------+
|         business_id|     category_name|
+--------------------+------------------+
|0x407636ffaaab494...|        Restaurant|
|0x407636ffaaab494...|Restaurant or cafe|
|0x4ccae17c7ed5a88...|        Restaurant|
+--------------------+------------------+
only showing top 3 rows

